# LangChain 기능 소개

- 1. 프롬프트 템플릿과 로더를 사용하여 체인 구성  
- 2. Runnable과 LangChain 표현 언어  
- 3. RAG 체인 구축  
- 4. 체인이 달린 도구 사용  

## LangChain 개요

LangChain으로 개발된 애플리케이션은 일반적으로 세 가지 범주로 나뉩니다.

- 챗봇 : LLM을 이용하여 보다 지능적인 마케팅, 고객 지원, 교육 상호작용 구현.
- 검색 증강 생성(RAG) Q&A : 대용량 문서 요약, 데이터 분석, 외부 소스를 참조하여 코드 생성에 사용.
- 에이전트 시스템은 다중 에이전트 설정과 인간 상호작용을 포함하며 복잡한 워크플로우를 위해 LangGraph를 활용. 공급망 관리 및 운영 최적화와 같은 분야에 적용 가능.

LangChain은 자연어를 실행 가능한 프로그램으로 변환하는 파이프라인을 생성할 수 있게 합니다. 이러한 체인을 활용함으로써 사용자는 자연어를 입력하고 보고서, 분석 또는 컴퓨터 프로그램과 같은 출력을 받을 수 있습니다.

In [1]:
# !pip install -qU \
# python-dotenv \
# langchain \
# langchain-community \
# openai \
# anthropic \
# langchain-openai \
# langchain-anthropic \
# langchain-google-genai

In [25]:
# env 파일에서 API 키를 로드
from dotenv import load_dotenv
load_dotenv()

True

### 사용 가능한 LLM 목록 조회

In [26]:
import openai

# 사용 가능한 모델 목록을 가져옵니다.
models = openai.models.list()

# 각 모델의 ID 출력
print([model.id for model in models])

['gpt-4o-audio-preview-2024-10-01', 'gpt-4o-realtime-preview', 'gpt-4o-realtime-preview-2024-10-01', 'dall-e-2', 'gpt-4-turbo', 'gpt-4-1106-preview', 'gpt-3.5-turbo', 'gpt-3.5-turbo-0125', 'gpt-3.5-turbo-instruct', 'babbage-002', 'whisper-1', 'dall-e-3', 'text-embedding-3-small', 'gpt-3.5-turbo-16k', 'gpt-4-0125-preview', 'gpt-4-turbo-preview', 'chatgpt-4o-latest', 'omni-moderation-latest', 'gpt-4o-2024-05-13', 'o1-preview-2024-09-12', 'omni-moderation-2024-09-26', 'tts-1-hd-1106', 'o1-preview', 'gpt-4', 'gpt-4-0613', 'tts-1-hd', 'text-embedding-ada-002', 'gpt-3.5-turbo-1106', 'gpt-4o-audio-preview', 'tts-1', 'tts-1-1106', 'gpt-3.5-turbo-instruct-0914', 'davinci-002', 'text-embedding-3-large', 'gpt-4o-realtime-preview-2024-12-17', 'gpt-4o-mini-realtime-preview', 'gpt-4o-mini-realtime-preview-2024-12-17', 'o1-mini', 'gpt-4o-2024-11-20', 'o1-mini-2024-09-12', 'gpt-4o-audio-preview-2024-12-17', 'gpt-4o-mini-audio-preview', 'gpt-4o-mini-2024-07-18', 'gpt-4o-mini', 'gpt-4o-2024-08-06', 'gpt

In [27]:
import anthropic
client = anthropic.Anthropic()
print([model.id for model in client.models.list()])

['claude-3-5-sonnet-20241022', 'claude-3-5-haiku-20241022', 'claude-3-5-sonnet-20240620', 'claude-3-haiku-20240307', 'claude-3-opus-20240229', 'claude-3-sonnet-20240229', 'claude-2.1', 'claude-2.0']


In [28]:
import google.generativeai as genai
print([model.name for model in genai.list_models()])

['models/chat-bison-001', 'models/text-bison-001', 'models/embedding-gecko-001', 'models/gemini-1.0-pro-latest', 'models/gemini-1.0-pro', 'models/gemini-pro', 'models/gemini-1.0-pro-001', 'models/gemini-1.0-pro-vision-latest', 'models/gemini-pro-vision', 'models/gemini-1.5-pro-latest', 'models/gemini-1.5-pro-001', 'models/gemini-1.5-pro-002', 'models/gemini-1.5-pro', 'models/gemini-1.5-pro-exp-0801', 'models/gemini-1.5-pro-exp-0827', 'models/gemini-1.5-flash-latest', 'models/gemini-1.5-flash-001', 'models/gemini-1.5-flash-001-tuning', 'models/gemini-1.5-flash', 'models/gemini-1.5-flash-exp-0827', 'models/gemini-1.5-flash-002', 'models/gemini-1.5-flash-8b', 'models/gemini-1.5-flash-8b-001', 'models/gemini-1.5-flash-8b-latest', 'models/gemini-1.5-flash-8b-exp-0827', 'models/gemini-1.5-flash-8b-exp-0924', 'models/gemini-2.0-flash-exp', 'models/gemini-exp-1206', 'models/gemini-exp-1121', 'models/gemini-exp-1114', 'models/gemini-2.0-flash-thinking-exp', 'models/gemini-2.0-flash-thinking-exp

## LLM 연결

OpenAI 및 Anthropic API에 연결합니다. 원하는 모델을 지정하여 ChatOpenAI 및 ChatAnthropic 클래스의 인스턴스를 만듭니다. llm_claude3 인스턴스를 사용하여 간단한 쿼리를 호출하여 설정을 확인합니다.

In [29]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI

llm_gpt4 = ChatOpenAI(model="gpt-4o-mini")

llm_claude3 = ChatAnthropic(model="claude-3-5-haiku-20241022")

llm_gemini = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [30]:
print(llm_gpt4.invoke("한국어로 LangChain 에 대해서 설명해줘").content)

LangChain은 자연어 처리(NLP)와 관련된 기능을 구축하는 데 도움을 주는 프레임워크입니다. 주로 대화형 AI 애플리케이션을 개발하는 데 사용되며, 다양한 언어 모델과 데이터 소스를 연결하여 복잡한 언어 처리 작업을 수행할 수 있게 해줍니다. LangChain은 다음과 같은 주요 기능을 제공합니다:

1. **체인(chain)**: 여러 개의 언어 모델 또는 처리 단계를 연결하여 복잡한 작업을 수행할 수 있도록 합니다. 예를 들어, 특정 질문에 대한 답변을 생성하기 위해 여러 단계를 거칠 수 있습니다.

2. **프롬프트 템플릿**: 사용자 요청에 대한 응답을 생성하기 위해 사용할 수 있는 템플릿을 제공합니다. 이를 통해 일관된 형식의 응답을 쉽게 생성할 수 있습니다.

3. **기억(memory)**: 대화의 맥락을 유지하여 이전 대화 내용을 기억하고, 이를 바탕으로 자연스럽고 연속적인 대화를 할 수 있도록 지원합니다.

4. **데이터 소스 통합**: 다양한 데이터 소스(예: 데이터베이스, API 등)와 연결하여 실시간으로 정보를 검색하고 사용할 수 있습니다.

5. **유연한 아키텍처**: 개발자가 필요에 따라 다양한 언어 모델과 알고리즘을 쉽게 통합하고 조정할 수 있는 유연성을 제공합니다.

LangChain은 대화형 시스템, 챗봇, 개인 비서 등 다양한 애플리케이션에서 활용될 수 있으며, 개발자들이 보다 쉽게 고급 언어 처리 기능을 구현할 수 있도록 도와줍니다.


In [31]:
print(llm_claude3.invoke("한국어로 LangChain 에 대해서 설명해줘").content)

LangChain은 대규모 언어 모델(LLM)을 활용한 애플리케이션 개발을 간소화하는 오픈 소스 Python 라이브러리입니다. 주요 특징은 다음과 같습니다:

1. 주요 기능
- LLM 애플리케이션 쉬운 개발
- AI 체인 및 에이전트 생성
- 다양한 LLM 모델 통합 지원
- 프롬프트 관리
- 메모리 및 상호작용 추적

2. 주요 구성 요소
- Prompts: 언어 모델 입력 관리
- Chains: 복잡한 작업 연결
- Agents: 자율적인 의사결정 및 작업 수행
- Memory: 대화 상태 유지
- Tools: 외부 리소스 연결

3. 지원 모델
- OpenAI GPT
- Hugging Face
- Google Vertex AI
- Anthropic Claude

4. 주요 활용 사례
- 대화형 챗봇
- 문서 분석
- 데이터 추출
- 코드 생성
- 질의응답 시스템

Python에서 쉽고 강력한 AI 애플리케이션 개발을 지원합니다.


In [32]:
print(llm_gemini.invoke("한국어로 LangChain 에 대해서 설명해줘").content)

LangChain은 대규모 언어 모델(LLM)을 기반으로 응용 프로그램을 구축하기 위한 프레임워크입니다.  단순히 LLM을 호출하는 것 이상으로, 여러 LLM과 다른 유형의 데이터 소스를 연결하여 더욱 복잡하고 강력한 애플리케이션을 만들 수 있도록 도와줍니다.  핵심 기능은 다음과 같습니다.

* **모듈화:** LangChain은 다양한 구성 요소(LLM, Prompts, 메모리, 인덱서 등)를 모듈화하여 재사용성과 확장성을 높였습니다.  각 구성 요소는 독립적으로 개발 및 교체될 수 있어 유연성이 뛰어납니다.

* **LLM 연결:** 다양한 LLM(OpenAI, Hugging Face, etc.)을 쉽게 연결하여 사용할 수 있습니다.  특정 LLM에 종속되지 않고 필요에 따라 LLM을 변경할 수 있습니다.

* **데이터 연결:**  LLM을 단순히 질문-응답에만 사용하는 것이 아니라, 데이터베이스, 파일 시스템, 웹 페이지 등 다양한 데이터 소스와 연결하여 정보를 활용할 수 있습니다.  이는 LLM의 지식 범위를 확장하고, 최신 정보를 활용하는 애플리케이션을 구축하는 데 필수적입니다.

* **메모리 관리:**  대화형 애플리케이션에서 이전 대화 내용을 기억하고 활용하는 메모리 기능을 제공합니다.  이는 일관성 있는 대화 흐름을 유지하는 데 중요합니다.

* **체인(Chains):** 여러 구성 요소를 연결하여 복잡한 작업을 수행하는 체인을 만들 수 있습니다. 예를 들어, 문서를 요약하고, 요약된 내용을 기반으로 질문에 답하는 체인을 구축할 수 있습니다.

* **에이전트(Agents):**  자율적으로 작업을 수행하는 에이전트를 구축할 수 있습니다.  에이전트는 여러 도구(LLM, 검색 엔진, 계산기 등)를 사용하여 목표를 달성합니다.  예를 들어, 에이전트는 사용자의 요청에 따라 웹을 검색하고, 정보를 수집하여 요약을 제공할 수 있습니다.


**LangChain을 사용하면 다음과 같은 애플리케이션을 개발할 수 있습니다.**

* **챗봇:**  

### Messages
- AIMessage: AI(인공지능) 모델이 생성한 메시지를 나타냅니다.  
예를 들어, AI 모델이 사용자의 질문에 대해 응답을 생성할 때 이 클래스가 사용됩니다.

- HumanMessage: 사람(사용자)이 생성한 메시지를 나타냅니다.  
사용자가 입력한 텍스트나 질문 등이 여기에 해당됩니다.

- StemMessage: AI 행동을 프라이밍하기 위한 메시지.
시스템 메시지는 일반적으로 일련의 입력 메시지 중 첫 번째로 전달됩니다. 일반적으로 대화의 흐름을 제어하거나 특정 지침을 제공하기 위해 사용됩니다.

** 프라이밍(Priming)은 시스템 메시지에서 특정 행동이나 반응을 유도하기 위해 미리 정보를 제공하거나 맥락을 설정하는 방법

In [18]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

system_prompt = """
마치 다섯살 먹은 어린아이에게 설명하듯이 한국어로 쉽게 설명해 주세요.
"""

user_prompt = """
LangChain이 무엇인가요?
"""

In [19]:
# system과 human/user 메시지를 이용한 기본 요청

message = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_prompt),
]

response = llm_gpt4.invoke(message)
response

AIMessage(content='LangChain은 컴퓨터가 사람처럼 말을 하고 생각할 수 있게 도와주는 특별한 도구예요. 우리가 친구와 이야기할 때, 어떤 주제에 대해 대화하거나 질문을 하는 것처럼, LangChain은 컴퓨터가 그런 대화를 잘 할 수 있도록 만들어 줘요. \n\n예를 들어, LangChain을 사용하면 컴퓨터가 책을 읽고, 그 내용을 이해하고, 우리에게 질문에 대한 대답을 할 수 있게 해줘요. 마치 친구와 이야기하는 것처럼요! 그래서 사람들이 필요로 하는 정보를 쉽게 찾고, 대화할 수 있도록 도와주는 역할을 해요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 135, 'prompt_tokens': 43, 'total_tokens': 178, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-c613840d-b7d9-42f7-9455-ca0c619d9e1a-0', usage_metadata={'input_tokens': 43, 'output_tokens': 135, 'total_tokens': 178, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reason

In [20]:
print(response.content)

LangChain은 컴퓨터가 사람처럼 말을 하고 생각할 수 있게 도와주는 특별한 도구예요. 우리가 친구와 이야기할 때, 어떤 주제에 대해 대화하거나 질문을 하는 것처럼, LangChain은 컴퓨터가 그런 대화를 잘 할 수 있도록 만들어 줘요. 

예를 들어, LangChain을 사용하면 컴퓨터가 책을 읽고, 그 내용을 이해하고, 우리에게 질문에 대한 대답을 할 수 있게 해줘요. 마치 친구와 이야기하는 것처럼요! 그래서 사람들이 필요로 하는 정보를 쉽게 찾고, 대화할 수 있도록 도와주는 역할을 해요.


In [21]:
# import textwrap

# #텍스트를 지정된 너비에 맞게 줄 바꿈하여 하나의 문자열로 반환
# answer = textwrap.fill(response.content, width=100)
# print(answer)

In [22]:
response = llm_claude3.invoke(message)

print(response.content)

LangChain을 아주 쉽게 설명해볼게요! 🤖

LangChain은 마치 레고 블록 같은 프로그램이에요. 

인공지능(AI) 언어 모델을 쉽고 재미있게 만들 수 있도록 도와주는 특별한 도구예요. 마치 레고로 다양한 모양을 만들 수 있듯이, LangChain으로 여러 가지 멋진 AI 기능을 만들 수 있답니다.

예를 들어:
• 챗봇 만들기
• 문서 요약하기
• 질문에 답변하기
• 복잡한 AI 작업 수행하기

쉽게 말해, AI 프로그래밍을 훨씬 더 간단하고 재미있게 만들어주는 마법 상자라고 생각하면 돼요! 🧙‍♂️✨


### 수학 선생님

In [23]:
messages = [
    SystemMessage(content="당신은  수학 문제를 잘 풀어주는 도움되는 assistant 입니다."),
    HumanMessage(content="81을 9로 나누면 몇인가요?")
]

result = llm_gpt4.invoke(messages)
print(result.content)

81을 9로 나누면 9입니다. (81 ÷ 9 = 9)


In [24]:
messages = [
    SystemMessage(content="당신은  수학 문제를 잘 풀어주는 도움되는 assistant 입니다."),
    HumanMessage(content="81을 9로 나누면 몇인가요?"),
    AIMessage(content="81을 9로 나누면 9입니다."),
    HumanMessage(content="10 곱하기 5는 얼마인가요?")
]

result = llm_gpt4.invoke(messages)
print(result.content)

10 곱하기 5는 50입니다.


### Chat Model을 이용한 Conversation 구현

In [16]:
# 채팅 기록을 저장할 리스트를 초기화
chat_history = []

# 시스템 메시지를 생성하여 채팅 기록에 추가
system_message = SystemMessage(content="You are a helpful AI assistant.")
chat_history.append(system_message)

# 무한 루프 시작
while True:
    # 사용자로부터 입력을 받음
    query = input("사용자: ")
    
    # 사용자가 "quit"를 입력하면 루프를 종료
    if query.lower() == "quit":
        break
    
    # 사용자 메시지를 채팅 기록에 추가
    chat_history.append(HumanMessage(content=query))

    # llm_gpt4 모델을 사용하여 응답을 생성
    result = llm_gpt4.invoke(chat_history)
    response = result.content
    
    # AI 응답을 채팅 기록에 추가
    chat_history.append(AIMessage(content=response))
    
    # AI 응답을 출력
    print(f"AI: {response}")

# 전체 대화 기록을 출력
print("대화 History:")
print(chat_history)

사용자:  안녕 내 이름은 오영제야


AI: 안녕하세요, 오영제님! 만나서 반갑습니다. 어떻게 도와드릴까요?


사용자:  강남 일원역에서 도봉산역을 가려면 어디서 갈아타야 하나요?


AI: 강남 일원역에서 도봉산역으로 가시려면 다음과 같이 이동하시면 됩니다:

1. **일원역**에서 3호선(수도권 전철)으로 탑승합니다.
2. 3호선을 타고 **교대역**에서 하차합니다.
3. 교대역에서 2호선으로 갈아탑니다.
4. 2호선을 타고 **신림역**으로 가서 하차합니다.
5. 신림역에서 7호선으로 갈아탑니다.
6. 7호선을 타고 **도봉산역**으로 이동합니다.

이 방법으로 이동하시면 도봉산역에 도착할 수 있습니다. 안전한 여행 되세요!


사용자:  그런데 내 이름이 뭐야?


AI: 오영제님이라고 하셨습니다! 맞나요? 다른 질문이나 도움이 필요하시면 언제든지 말씀해 주세요.


사용자:  quit


대화 History:
[SystemMessage(content='You are a helpful AI assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕 내 이름은 오영제야', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요, 오영제님! 만나서 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='강남 일원역에서 도봉산역을 가려면 어디서 갈아타야 하나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='강남 일원역에서 도봉산역으로 가시려면 다음과 같이 이동하시면 됩니다:\n\n1. **일원역**에서 3호선(수도권 전철)으로 탑승합니다.\n2. 3호선을 타고 **교대역**에서 하차합니다.\n3. 교대역에서 2호선으로 갈아탑니다.\n4. 2호선을 타고 **신림역**으로 가서 하차합니다.\n5. 신림역에서 7호선으로 갈아탑니다.\n6. 7호선을 타고 **도봉산역**으로 이동합니다.\n\n이 방법으로 이동하시면 도봉산역에 도착할 수 있습니다. 안전한 여행 되세요!', additional_kwargs={}, response_metadata={}), HumanMessage(content='그런데 내 이름이 뭐야?', additional_kwargs={}, response_metadata={}), AIMessage(content='오영제님이라고 하셨습니다! 맞나요? 다른 질문이나 도움이 필요하시면 언제든지 말씀해 주세요.', additional_kwargs={}, response_metadata={})]


##  1. 프롬프트 템플릿과 로더를 사용하여 체인 구성  

- LLM에서의 Chain 은 Data Processing 에서의 Pipeline 과 유사한 개념

### 체인의 구성 요소 - Runnables
- 프롬프트 : LLM의 응답을 안내하는 템플릿  
- LLM 또는 채팅 모델 : 프롬프트에 따라 응답을 생성하는 엔진  
- 출력 파서 : LLM의 출력을 파싱하는 도구  
- 도구 : LLM이 API에서 추가 정보를 추출하거나 코드를 실행하여 LLM을 에이전트로 전환할 수 있게 해주는 확장 기능  
- 일반 함수 : 서로 연결될 수 있는 추가적인 일반 함수

In [41]:
from langchain.prompts import PromptTemplate

# simple prompt template 생성
# {topic} 변수에서 사용자의 query 가 대체된다.
prompt_template = """
    당신은 AI 주제를 설명하는 데 도움이 되는 어시스턴트입니다. 다음 입력이 주어지면:
    {주제}
    주어진 주제에 대한 설명을 제공하세요.
"""

# prompt template 을 이용하여 prompt 생성
prompt = PromptTemplate(
    input_variables=["주제"],
    template=prompt_template,
)

In [42]:
# pipe operator "|"" 를 이용하여 하나 이상의 chain 을 결합
chain = prompt | llm_gpt4

print(chain.invoke({"주제": "LangChain이 뭐예요?"}).content)

LangChain은 자연어 처리(NLP)와 관련된 애플리케이션을 구축하기 위한 프레임워크입니다. 이 프레임워크는 다양한 언어 모델과 API를 쉽게 통합할 수 있도록 설계되어 있어, 개발자들이 언어 모델을 활용한 복잡한 애플리케이션을 보다 간편하게 만들 수 있도록 도와줍니다.

LangChain의 주요 구성 요소는 다음과 같습니다:

1. **문서 관리**: LangChain은 문서를 수집하고 이를 처리하는 데 필요한 다양한 도구를 제공합니다. 이를 통해 대량의 텍스트 데이터를 효과적으로 관리할 수 있습니다.

2. **프롬프트 관리**: 사용자가 언어 모델에 전달할 프롬프트를 쉽게 생성하고 관리할 수 있도록 돕습니다. 이는 모델의 응답 품질을 향상시키는 데 중요한 역할을 합니다.

3. **체인**: LangChain은 여러 작업을 순차적으로 연결하여 복잡한 프로세스를 구성할 수 있는 체인 구조를 제공합니다. 이를 통해 데이터 흐름을 명확하게 정의할 수 있습니다.

4. **모델 통합**: 다양한 언어 모델(예: OpenAI의 GPT, Hugging Face의 모델 등)과 쉽게 통합할 수 있는 기능을 제공합니다. 개발자는 특정 모델의 특성에 맞춰 애플리케이션을 최적화할 수 있습니다.

5. **응용 프로그램 템플릿**: LangChain은 다양한 유형의 NLP 애플리케이션을 구축할 수 있는 템플릿을 제공하여, 개발자들이 신속하게 프로토타입을 만들고 테스트할 수 있도록 합니다.

LangChain은 이처럼 다양한 기능을 통해 자연어 처리 관련 프로젝트의 개발 속도를 높이고, 효율성을 향상시키는 데 기여합니다. 이를 통해 개발자들은 더욱 창의적이고 혁신적인 NLP 솔루션을 개발할 수 있습니다.


### document loader 를 이용하여 script 요약
더 고급 체인을 만들기 위해 LangChain을 사용하여 YouTube 비디오를 필사합니다. 먼저 YouTube Transcript API를 설치합니다. 그런 다음 LangChain의 커뮤니티 문서 로더에서 YouTube Loader를 가져옵니다. 로더에 비디오 URL을 전달하면 필사본을 추출할 수 있으며, 이는 문서 목록으로 반환됩니다. 원시 필사본을 얻으려면 이러한 문서에서 페이지 콘텐츠를 추출하거나 문서를 체인에 직접 전달하기만 하면 됩니다.

In [43]:
# !pip install --upgrade --quiet youtube-transcript-api

In [52]:
# LangChain 커뮤니티에서 제공하는 Youtube Loader를 임포트
from langchain_community.document_loaders import YoutubeLoader

# YoutubeLoader를 사용하여 유튜브 URL에서 로더를 생성
loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=AOEGOhkGtjI", add_video_info=False
)

# 비디오의 자막을 문서로 로드
docs = loader.load()

In [46]:
transcript = docs[0].page_content
transcript

"so now we have Lama 3 from meta and this model is definitely going to be a GameChanger when it comes to analyzing data with llms here I have L 3 on gr cloud and not only are the text to SQL chains blazing fast they're also capable of generating quite Advanced SQL and almost on par with the high IQ llms if we Implement one additional tweak so in this video I'm going to show you how we can tweak the SQL chains to maximize the performance of Lama 3 we're going to have a look at some of the insights that Lama three is capable of extracting and finally I'm going to briefly discuss some of the implications for llm based data analysis on l.a. comom you can read about Lama 3 and some of the capabilities of the model you can see how the model compares to other popular llms specifically the 70b model that I'm going to be using in this video is compared to Gemini and CLA 3 Sunnet and there's also a link that lets you request access to Lama 3 but this is not what I'll be doing I'm going to be usi

이제, 대본이 주어진 YouTube 비디오를 요약하는 체인을 설정해 보겠습니다. 비디오 대본을 입력 변수로 프롬프트 템플릿에 주입한 다음 파이프 연산자를 사용하여 체인을 설정합니다. 비디오 대본으로 호출하면 원시 텍스트를 수동으로 추출하지 않고도 간결한 요약을 얻을 수 있습니다.

In [54]:
# 전사된 내용을 chain에 사용합니다.
# prompt template 생성
prompt_template = """
    당신은 유튜브 비디오를 요약하는 유용한 조수입니다. 다음 비디오 대본이 주어질 때:
    {video_transcript}
    한국어로 요약해 주세요.
"""

# prompt instance 생성
prompt = PromptTemplate(
    input_variables=["video_transcript"],
    template=prompt_template,
)
prompt

PromptTemplate(input_variables=['video_transcript'], input_types={}, partial_variables={}, template='\n    당신은 유튜브 비디오를 요약하는 유용한 조수입니다. 다음 비디오 대본이 주어질 때:\n    {video_transcript}\n    한국어로 요약해 주세요.\n')

In [55]:
# chain 생성
chain = prompt | llm_gpt4

answer = chain.invoke({"video_transcript": docs}).content

In [56]:
print(answer)

이 비디오는 Meta의 Lama 3 모델을 사용하여 데이터 분석을 위한 SQL 체인을 최적화하는 방법을 다룹니다. 발표자는 Gr Cloud에서 Lama 3를 설정하고, SQL 생성 성능을 극대화하기 위한 조정 방법을 설명합니다. 사용자는 e-commerce 데이터 세트를 활용하여 고객 정보 및 수익을 분석하는 여러 SQL 쿼리를 생성합니다.

주요 내용은 다음과 같습니다:
1. **모델 소개**: Lama 3는 빠르고 고급 SQL 생성이 가능하며, 다른 고급 LLM과 비교할 수 있는 성능을 보여줍니다.
2. **SQL 체인 설정**: 필요한 라이브러리를 설치하고, 데이터 세트의 스키마 정보를 추출하여 SQL 체인을 설정합니다.
3. **자기 수정 기능**: SQL 코드 생성 시 오류 발생 시 이를 체인에 피드백하여 자동으로 수정하는 기능을 구현합니다.
4. **쿼리 실행 예시**: 여러 가지 쿼리를 시도하여 성공적인 결과를 도출하며, 오류를 피드백하여 성과를 개선하는 과정을 보여줍니다.
5. **미래의 데이터 분석**: Lama 3와 같은 LLM이 데이터 파이프라인, 대시보드 및 보고서 생성에 중요한 역할을 할 것으로 예상되며, 데이터 분석가와 엔지니어는 이러한 기술을 익힐 필요가 있음을 강조합니다.

비디오는 Lama 3의 잠재력과 LLM 기반 데이터 분석의 향후 방향성을 제시하며 마무리됩니다.


### Built-in Chains 이용 예

- create_stuff_documents_chain : 모델에 문서 목록을 전달하기 위한 체인을 생성

In [57]:
from langchain.chains.combine_documents import create_stuff_documents_chain

# create_stuff_documents_chain은 문서 목록을 받아 모두 하나의 프롬프트로 형식화합니다.
# create_stuff_documents_chain 의 input variable 이름은 context 여야 합니다.

prompt_template = """
    당신은 유튜브 비디오를 요약하는 유용한 조수입니다. 다음 비디오 대본이 주어질 때:
    {context}
    한국어로 요약해 주세요.
"""

# prompt instance 생성
prompt = PromptTemplate(
    input_variables=["context"],
    template=prompt_template,
)

# chain 생성
chain = create_stuff_documents_chain(llm_gpt4, prompt)

answer = chain.invoke({"context": docs})

In [58]:
print(answer)

이 비디오는 메타의 새로운 모델인 Lama 3에 대해 다룹니다. Lama 3는 데이터 분석에 있어 큰 변화를 가져올 것으로 기대되며, 특히 SQL 생성에서 빠르고 정교한 성능을 보여준다고 합니다. 이 비디오에서는 SQL 체인을 최적화하는 방법과 Lama 3가 제공하는 인사이트를 살펴보며, LLM 기반 데이터 분석의 의미에 대해서도 논의합니다.

비디오에서는 Google Cloud와 LangChain을 사용하여 SQL 체인을 설정하는 과정이 설명됩니다. 사용자가 데이터셋의 스키마 정보를 추출하고, 이를 기반으로 SQL 쿼리를 생성하는 과정을 보여줍니다. 오류가 발생할 경우 이를 자동으로 수정하는 기능도 강조되며, 여러 번의 시도 끝에 성공적인 쿼리를 얻는 과정을 예시로 들고 있습니다.

Lama 3의 활용 가능성에 대해 논의하면서, 개인 정보 보호가 중요한 상황에서의 사용이나 대규모 쿼리 애플리케이션에서의 비용 문제 등을 언급합니다. 마지막으로, 데이터 분석가 및 데이터 엔지니어는 이러한 새로운 기술에 주목해야 한다고 강조하며 비디오를 마무리합니다.


## 2. Runnable과 LangChain 표현 언어  

- LCEL(LangChain Expression Language) 는 Runnables 를 chain 으로 구성하는 방법

LCEL은 기본 구성 요소에서 복잡한 체인을 구축하는 것을 간소화합니다. 파이프 연산자(|)를 사용하여 다양한 구성 요소를 체인으로 연결하고 한 요소에서 다음 요소로 출력을 공급합니다. 이런 방식으로 구성된 체인의 간단한 예로는 모델과 출력 파서가 결합된 프롬프트가 있습니다.  이러한 구성 요소들을 runnables 라고 부릅니다.  

`chain=prompt | model | output_parser`

- Chain 구성  
    - 체인에 대한 입력(일반적으로 사전)
    - 입력은 프롬프트로 전송됩니다.
    - 프롬프트 값은 LLM 또는 채팅 모델로 전송됩니다.
    - Chatmodel이 채팅 메시지를 반환합니다.
    - 파서는 채팅 메시지에서 문자열을 추출합니다.
    - 문자열은 체인의 출력입니다

- LangChain의 runnable 객체들:

    - RunnableSequence : 여러 runnable 구성 요소를 연결하여 각 구성 요소가 입력을 처리하고 출력을 다음 구성 요소에 전달
    - RunnableLambda : Python의 호출 가능한 요소(함수 등)를 실행 가능한 구성 요소로 바꿔서 체인으로 통합
    - RunnablePassthrough : 입력을 변경하지 않고 통과시키거나 출력에 추가 키를 추가. placeholder 역할을 하거나 시퀀스에 유연하게 통합할 수 있다.
    - RunnableParallel : 여러 개의 실행 파일을 동시에 실행하여 두 개의 체인이 동일한 입력에서 실행되지만 다른 출력을 반환하는 분기를 허용.

### Runnables

In [59]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

summarize_prompt_template = """
    당신은 AI 개념을 요약하는 유용한 조수입니다.
    {context}
    context를 한국어로 요약해 주세요.
"""

summarize_prompt = PromptTemplate.from_template(summarize_prompt_template)

###  "|" 연산자를 이용하여 Runnable Sequence chain 생성

In [60]:
output_parser = StrOutputParser()

chain = summarize_prompt | llm_gpt4 | output_parser

print(type(chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [61]:
chain.invoke({"context": "LangChain 에 대해 설명해 줘."})

'LangChain은 자연어 처리(NLP) 모델을 기반으로 다양한 애플리케이션을 구축할 수 있도록 돕는 프레임워크입니다. 이 프레임워크는 언어 모델을 활용하여 정보 검색, 대화형 에이전트, 문서 분석 등 다양한 작업을 수행할 수 있는 도구와 구성 요소를 제공합니다. LangChain은 데이터를 처리하고, 사용자와 상호작용하며, 복잡한 언어 이해 작업을 수행하는 데 필요한 기능을 통합하여 개발자들이 쉽게 사용할 수 있도록 설계되었습니다.'

### RunnableLambda 사용

In [62]:
# RunnableLambda를 사용하여 Python 함수를 체인에 삽입합니다.
from langchain_core.runnables import RunnableLambda

summarize_chain = summarize_prompt | llm_gpt4 | output_parser
print(type(summarize_chain))

# 사용자 정의 람다 함수를 정의하고 이를 RunnableLambda에 래핑합니다.
length_lambda = RunnableLambda(lambda summary: f"요약된 길이: {len(summary)} 글자")
print(type(length_lambda))

lambda_chain = summarize_chain | length_lambda
print(type(lambda_chain))

<class 'langchain_core.runnables.base.RunnableSequence'>
<class 'langchain_core.runnables.base.RunnableLambda'>
<class 'langchain_core.runnables.base.RunnableSequence'>


In [63]:
lambda_chain.invoke({"context": "LangChain 에 대해 설명해 줘"})

'요약된 길이: 240 글자'

In [64]:
for step in lambda_chain.steps:
    print(step)

input_variables=['context'] input_types={} partial_variables={} template='\n    당신은 AI 개념을 요약하는 유용한 조수입니다.\n    {context}\n    context를 한국어로 요약해 주세요.\n'
client=<openai.resources.chat.completions.Completions object at 0x0000020DA72A9D90> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000020DA72B5F40> root_client=<openai.OpenAI object at 0x0000020DA54632E0> root_async_client=<openai.AsyncOpenAI object at 0x0000020DA72A9DF0> model_name='gpt-4o-mini' model_kwargs={} openai_api_key=SecretStr('**********')

RunnableLambda(lambda summary: f'요약된 길이: {len(summary)} 글자')


--------------
함수를 명시적으로 RunnableLambda로 변환하지 않고도 체인에서 함수를 사용할 수 있습니다

In [65]:
# RunnableLambda로 변환하지 않고 체인에서 함수 사용
chain_without_function = summarize_chain | (lambda summary: f"요약된 길이: {len(summary)} 글자")

chain_without_function.steps[-1]

RunnableLambda(lambda summary: f'요약된 길이: {len(summary)} 글자')

In [66]:
chain_without_function.invoke({"context": "LangChain 에 대해 설명해 줘"})

'요약된 길이: 244 글자'

### placeholder 로 사용되는 RunnablePassthrough

체인 내에서 데이터를 변경하지 않고 다음 단계로 전달하는 데 사용됩니다. 주로 디버깅하거나 체인의 특정 부분을 테스트할 때 유용합니다.

In [67]:
from langchain_core.runnables import RunnablePassthrough

summarize_chain = summarize_prompt | llm_gpt4 | output_parser

# RunnablePassthrough instance 생성
passthrough = RunnablePassthrough()

# 요약 및 길이 계산과 함께 파이프 연산자를 사용하여 시퀀스 생성
placeholder_chain = summarize_chain | passthrough | length_lambda

placeholder_chain.invoke({"context": "LangChain 에 대해 설명해 줘"})

'요약된 길이: 230 글자'

In [68]:
placeholder_chain.steps

[PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    당신은 AI 개념을 요약하는 유용한 조수입니다.\n    {context}\n    context를 한국어로 요약해 주세요.\n'),
 ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000020DA72A9D90>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000020DA72B5F40>, root_client=<openai.OpenAI object at 0x0000020DA54632E0>, root_async_client=<openai.AsyncOpenAI object at 0x0000020DA72A9DF0>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********')),
 StrOutputParser(),
 RunnablePassthrough(),
 RunnableLambda(lambda summary: f'요약된 길이: {len(summary)} 글자')]

### RunnablePassthrough for assignment
- 데이터를 그대로 통과시키는 역할을 하면서 추가적으로 특정 키에 값을 할당
- 체인을 통과하는 데이터에 "요약" 필드의 길이를 계산하고 length라는 키로 추가

In [69]:
# 요약을 dictionary로 변환하는 사용자 정의 람다 함수 정의
wrap_summary_lambda = RunnableLambda(lambda summary: {"요약": summary})

# Dict 입력에 mapping argument를 merge RunnablePassthrough 인스턴스 생성
assign_passthrough = RunnablePassthrough.assign(length=lambda x: len(x['요약']))

# summarization chain 생성
summarize_chain = summarize_prompt | llm_gpt4 | output_parser | wrap_summary_lambda

# 요약과 assign_passthrough를 결합하여 전체 체인을 생성
assign_chain = summarize_chain | assign_passthrough

# Use the chain
result = assign_chain.invoke({"context": "LangChain 에 대해 설명해 줘"})
result

{'요약': 'LangChain은 자연어 처리(NLP) 모델과 애플리케이션을 구축하는 데 도움을 주는 프레임워크입니다. 이 프레임워크는 대화형 AI 시스템, 챗봇, 검색 시스템 등 다양한 용도로 활용될 수 있습니다. LangChain은 데이터 소스와 통합하여 언어 모델을 활용할 수 있도록 하며, 사용자 정의 로직과 다양한 API를 결합해 복잡한 작업을 수행할 수 있게 해줍니다. 또한, LangChain은 모듈화된 구성 요소를 제공하여 개발자가 필요에 맞게 시스템을 쉽게 확장하고 조정할 수 있도록 합니다.',
 'length': 276}

In [70]:
print(type(assign_chain.steps[-1]))

<class 'langchain_core.runnables.passthrough.RunnableAssign'>


### RunnableParallel 
- 체인의 특정 단계에서 입력을 동시에 여러 작업에 전달하고 각각의 결과를 수집하여 반환합니다

In [71]:
from langchain_core.runnables import RunnableParallel

#summarization chain 생성
summarization_chain = summarize_prompt | llm_gpt4 | output_parser

# summary 와 길이 계산을 병렬로 처리하기 위해 RunnableParallel 인스턴스를 만듭니다.
parallel_runnable = RunnableParallel(
    summary=lambda x: x,  # 현재의 summary 전달
    length=lambda x: len(x)   # summary의 길이 계산
)

# parellel runnable과 summarization 결합
parallel_chain = summarize_chain | parallel_runnable

parallel_chain.invoke({"context": "LangChain 에 대해 설명해 줘"})

{'summary': {'요약': 'LangChain은 자연어 처리(NLP)와 관련된 다양한 애플리케이션을 개발하기 위한 프레임워크입니다. 이 프레임워크는 언어 모델을 효율적으로 사용하고, 여러 데이터 소스와 통합하며, 복잡한 작업을 수행할 수 있도록 지원합니다. LangChain은 모듈화된 구성 요소를 제공하여 개발자가 자연어 처리 시스템을 쉽게 구축하고 확장할 수 있도록 돕습니다. 예를 들어, 언어 모델, 데이터베이스, API 등을 연결하여 더 풍부한 기능을 가진 애플리케이션을 만들 수 있습니다. LangChain은 특히 대화형 AI, 질문 응답 시스템, 텍스트 요약 등 다양한 분야에서 활용될 수 있습니다.'},
 'length': 1}

## 3. RAG Chain 만들기

RAG는 외부 데이터로 대형 언어 모델(LLM)의 지식을 증강하는 기술입니다. RAG 애플리케이션은 두 단계로 구축할 수 있습니다:   
   
1) 외부 데이터 색인화   
2) 검색 및 출력 생성.

### Choma DB 설치를 위해서는 Visual Studio Installer 에서 C++ 를 CHECK 하고 설치

In [46]:
# !pip install -U langchain-chroma

In [73]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import os

# directory 경로 정의
persistent_directory = "./db/chroma_db_test"
persistent_directory

'./db/chroma_db_test'

In [74]:
# Step 1: WebBaseLoader를 사용하여 apple.com에서 콘텐츠 스크래핑
# WebBaseLoader는 웹 페이지를 로드하고 해당 내용을 추출합니다.
urls = ["https://www.assembly.go.kr/portal/main/main.do"]

# 웹 콘텐츠에 대한 로더를 만듭니다
loader = WebBaseLoader(urls)
documents = loader.load()

# Step 2: 스크래핑된 콘텐츠를 청크로 분할
# CharacterTextSplitter는 텍스트를 더 작은 덩어리로 나눕니다.
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# 분할된 문서에 대한 정보 표시
print("\n--- Document Chunks Information ---")
print(f"Number of document chunks: {len(docs)}")
print(f"Sample chunk:\n{docs[0].page_content}\n")

Created a chunk of size 4335, which is longer than the specified 1000
Created a chunk of size 5962, which is longer than the specified 1000



--- Document Chunks Information ---
Number of document chunks: 13
Sample chunk:
대한민국 국회 홈페이지

오늘열지않기

닫기


오늘열지않기

닫기


팝업건수: 총 0건

닫기
오늘하루 열지않기


오늘 하루 열지 않기


창닫기


본문내용 바로가기


찾으시는 검색어를 입력해주세요

통합검색


검색

창닫기

전체메뉴

대한민국국회
내일을 여는 국민의 국회

페이스북
유투브
트위터
인스타그램
RSS

국회활동


국회회의


국회일정
의사일정 공지
주요정치일정 공지
회의록
인터넷의사중계
영상회의록
국회경과보고서

의안


의안현황
의안안내

예산안과 결산


예/결산 현황
예/결산 안내

국정감사


국정감사 현황
국정감사제도 안내

의회외교


의회외교개요
의회외교단체
초청외교
방문외교
국제회의

국회공보


국회공보

의원활동


국회의장


국회의장 소개
주요일정
주요동정
보도자료
연설문
역대 국회의장

국회부의장


국회부의장 소개
주요일정
주요동정
보도자료

국회의원현황


국회의원현황
역대 국회의원현황
국회의원 안내

의원실행사

의원실채용


의원연구단체


의원연구단체
연구단체 안내

친인척보좌직원 현황

위원회


위원회현황


위원회현황
위원회안내

위원명단


위원명단
위원명단공지

위원회일정

계류의안

위원회회의록

위원회자료실

공시송달

소통마당


국회민원


국회민원
민원안내

국민동의청원


국민동의청원 현황
국민동의청원 안내

입법예고


입법예고
입법예고 안내

열린국회정보


정보공개청구

국회박물관

국회참관

문화행사

의정연수


국민제안

알림마당


공지사항

보도자료

행사알림

국회 SNS 소식


국회공식 SNS 소식
국회의원 SNS 소식
국회 SNS

국회매거진

국회정보길라잡이


정책참고자료실

국회채용

분실물 안내

국회소개


국회의 구성


국회의장단
국회의원
위원회
교섭단체
입법지원조직
조직도

국회가 하는 일


역할과 권

In [76]:
# Step 3: 문서 청크에 대한 임베딩을 만듭니다.
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Step 4: 임베딩을 사용하여 벡터 저장소를 생성하고 유지합니다.
if not os.path.exists(persistent_directory):
    print(f"\n--- {persistent_directory}에 벡터 저장소 생성 ---")
    db = Chroma.from_documents(
        docs, embeddings, persist_directory=persistent_directory)
    print(f"--- {persistent_directory}에 벡터 저장소 생성 완료 ---")
else:
    print(f"벡터 저장소 {persistent_directory}가 이미 존재 합니다. 새로이 생성하지 않았습니다.")
    db = Chroma(persist_directory=persistent_directory, embedding_function=embeddings)

# Step 5: 벡터 저장소 쿼리
# 벡터 저장소를 쿼리하기 위한 검색기 생성
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)

벡터 저장소 ./db/chroma_db_test가 이미 존재 합니다. 새로이 생성하지 않았습니다.


In [77]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """
    다음 내용에만 근거하여 질문에 답하세요:
    {context}
    Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

output_parser = StrOutputParser()

In [78]:
chain = (
    # "context"는 입력 데이터에서 'question' 값을 가져오고, 이를 retriever에 전달하여 유사한 문서 검색
    {"context": (lambda x: x['question']) | retriever,
     "question": (lambda x: x['question'])}  # "question"은 입력 데이터에서 'question' 값을 그대로 반환
    | prompt    # 검색된 문서를 이용해 프롬프트를 생성함
    | llm_gpt4   # 프롬프트를 기반으로 GPT-4 모델을 사용해 응답을 생성
    | StrOutputParser()    # LLM의 출력을 문자열로 파싱
)

In [79]:
answer = chain.invoke({"question": "대한민국 국회가 주로 하는 일이 뭐야?"})
print(answer)

대한민국 국회는 대국민 서비스 제공, 민원 처리, 소관 업무 수행 등의 목적으로 활동하며, 주요 역할은 법률 제정, 예산 심의, 국정 감사, 의회 외교 등입니다. 국회는 국민의 대표로서 정책을 논의하고 결정하는 기관으로, 여러 위원회와 의원이 이러한 역할을 수행합니다.


## 4. 체인이 달린 도구 사용  

- 도구는 에이전트, 체인 또는 대형 언어 모델(LLM)이 세상과 상호 작용할 수 있게 하는 인터페이스입니다.
    - Python REPL, Wikipdedia, YouTube, Zapier, Gradio, etc

In [80]:
# !pip install --upgrade -q youtube_search

In [81]:
from langchain_community.tools import YouTubeSearchTool

# YouTubeSearchTool 인스턴스 생성
youtube_tool = YouTubeSearchTool()

# 아무 키워드로 유튜브 검색 실행
results = youtube_tool.run("코딩하는 AI")
print(results)

['https://www.youtube.com/watch?v=PL48_gUxKjQ&pp=ygUP7L2U65Sp7ZWY64qUIEFJ', 'https://www.youtube.com/watch?v=-ZRP1nnjRSo&pp=ygUP7L2U65Sp7ZWY64qUIEFJ']


In [82]:
# YouTube 도구를 LLM에 바인딩하여 parameter 정보를 알아냄
llm_with_tools = llm_gpt4.bind_tools([youtube_tool])

msg = llm_with_tools.invoke("코딩하는 AI")
msg

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_p2jOvNM8VLDwFucfTLtEDPi8', 'function': {'arguments': '{"query":"코딩하는 AI"}', 'name': 'youtube_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 94, 'total_tokens': 112, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-96c7cf0b-1ce3-44f2-bc1e-e3f3e3850f37-0', tool_calls=[{'name': 'youtube_search', 'args': {'query': '코딩하는 AI'}, 'id': 'call_p2jOvNM8VLDwFucfTLtEDPi8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 94, 'output_tokens': 18, 'total_tokens': 112, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'r

In [83]:
# llm_with_tools 에서 parameter 정보를 알아낸다
msg.tool_calls

[{'name': 'youtube_search',
  'args': {'query': '코딩하는 AI'},
  'id': 'call_p2jOvNM8VLDwFucfTLtEDPi8',
  'type': 'tool_call'}]

In [84]:
msg.tool_calls[0]["args"]

{'query': '코딩하는 AI'}

In [85]:
# llm_with_tools에서 추출된 인수로 
# YouTubeSearchTool을 사용하여 유튜브 검색 실행한는 chain 생성
chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | youtube_tool
chain

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000020DA72A9D90>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000020DA72B5F40>, root_client=<openai.OpenAI object at 0x0000020DA54632E0>, root_async_client=<openai.AsyncOpenAI object at 0x0000020DA72A9DF0>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'youtube_search', 'description': 'search for youtube videos associated with a person. the input to this tool should be a comma separated list, the first part contains a person name and the second a number that is the maximum number of video results to return aka num_results. the second part is optional', 'parameters': {'properties': {'query': {'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}]}, config={}, config_factories=[])
| RunnableLambda(lambda x: x.tool_calls[0]['args'])
| YouTubeSearchTool

In [86]:
response = chain.invoke("코딩하는 AI")

In [87]:
import ast

ast.literal_eval(response)

['https://www.youtube.com/watch?v=PL48_gUxKjQ&pp=ygUP7L2U65Sp7ZWY64qUIEFJ',
 'https://www.youtube.com/watch?v=-ZRP1nnjRSo&pp=ygUP7L2U65Sp7ZWY64qUIEFJ']

## 실습 : Prompt 수정하여 실행